<a href="https://colab.research.google.com/github/cuducquang/ML_Project/blob/main/task2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab Link
This cell provides a link to open the notebook in Google Colab, a cloud-based platform for running Jupyter Notebooks with GPU support. The badge ensures easy access to the notebook's environment.
- **Purpose**: Facilitates sharing and collaboration by linking directly to the Google Colab environment.
- **Output**: Displays a clickable badge to open the notebook in Colab.

### Install Keras Tuner
This cell installs the `keras-tuner` library, which is used for hyperparameter tuning of the neural network model.


In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.0 MB/s eta 0:00:00


## Import Libraries

### Import Libraries
This cell imports all necessary Python libraries and modules required for data processing, model building, training, and evaluation.

In [2]:
from google.colab import drive
import gdown
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import hashlib
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras_tuner import HyperParameters
from tensorflow import keras
import tensorflow as tf
from google.colab import files

## Download Training Dataset

### Download Training Dataset
This cell mounts Google Drive and downloads the training dataset (`train_images.zip`) from Google Drive using the `gdown` library.

In [3]:
drive.mount('/content/drive')

# Google Drive file ID (from the shared link)
file_id = "1l-KlchrHmf3v87KneUe1Ejn-u9Ubfi1S"
file_name = "train_images.zip"
file_path = "/content/" + file_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)
print(f"Downloaded: {file_path}")

Mounted at /content/drive


Downloading...
From (original): https://drive.google.com/uc?id=1l-KlchrHmf3v87KneUe1Ejn-u9Ubfi1S
From (redirected): https://drive.google.com/uc?id=1l-KlchrHmf3v87KneUe1Ejn-u9Ubfi1S&confirm=t&uuid=f518dc1e-2de1-48d4-b2ef-3468717b830b
To: /content/train_images.zip
100%|██████████| 517M/517M [00:06<00:00, 74.6MB/s]

Downloaded: /content/train_images.zip


## Unzip Training Dataset

### Unzip Training Dataset
This cell extracts the contents of the `train_images.zip` file to a specified directory.

In [4]:
!unzip -q $file_path -d /content/extracted_folder

## Data Augmentation for Training and Validation Datasets

### Data Augmentation for Training and Validation Datasets
This cell sets up data augmentation for the training dataset and prepares the validation dataset for model training.

In [5]:
train_dir = '/content/extracted_folder/train'
val_dir = '/content/extracted_folder/val'

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, # 224, 224 , 3
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

val_generator = val_test_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 21289 images belonging to 10 classes.
Found 2071 images belonging to 10 classes.


Purpose:
- Training Data Augmentation: Enhances the training dataset with transformations to improve model generalization.
- Validation Data Preparation: Normalizes validation images without augmentation.

Key Components:
- Training Directory: /content/extracted_folder/train contains training images.
- Validation Directory: /content/extracted_folder/val contains validation images.

ImageDataGenerator for Training:
- rescale=1.0/255.0: Normalizes pixel values to [0, 1].
- rotation_range=10: Rotates images by up to 10 degrees.
- width_shift_range=0.05, height_shift_range=0.05: Shifts images horizontally/vertically by 5%.
- zoom_range=0.1: Zooms in/out by 10%.
- horizontal_flip=True: Randomly flips images horizontally.
- brightness_range=[0.8, 1.2]: Adjusts brightness by 80% to 120%.
- fill_mode='nearest': Fills missing pixels with the nearest value.

Training Generator:
- Loads images from train_dir.
- Resizes images to 224x224 pixels (required by VGG16).
- Uses a batch size of 32.
- class_mode='categorical': One-hot encodes labels for 10 classes.

Validation Generator:
- Only rescales images (no augmentation).
- Uses a larger batch size of 64 for efficiency.

Output:
- Found 21289 images belonging to 10 classes. (Training)
- Found 2071 images belonging to 10 classes. (Validation)

Why It Matters: Data augmentation prevents overfitting by creating varied training samples, while the validation set evaluates model performance on unmodified data.

## Verify Training Generator Output

### Verify Training Generator Output
This cell checks the shape and content of a batch from the training generator to ensure correct data formatting.

In [6]:
x_batch, y_batch = next(train_generator)
print("Shape y_batch:", y_batch.shape)
print("Sample y_batch[0]:", y_batch[0])

Shape y_batch: (32, 10)
Sample y_batch[0]: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## Build VGG16 Architecture Model

### Build VGG16 Architecture Model
This cell defines a function to build a VGG16-based model with tunable hyperparameters and sets up hyperparameter tuning using `keras-tuner`.

##### Purpose: Defines a customizable VGG16 model and sets up hyperparameter tuning to optimize its architecture.
#### Key Components:
    + Model Architecture:
        + Uses VGG16 without pre-trained weights (weights=None) and without the top layers (include_top=False).
        + Input shape is (224, 224, 3) (RGB images).
        + Adds GlobalAveragePooling2D to reduce spatial dimensions.
        + Dense Block 1:
        + Dense layer with tunable units (256 to 1024, step 128).
        + BatchNormalization for stabilizing training.
        + ReLU activation.
        + Dropout with tunable rate (0.2 to 0.5, step 0.1).
        + Dense Block 2:
        + Dense layer with tunable units (64 to 512, step 64).
        + BatchNormalization and ReLU.
        + Dropout with tunable rate (0.1 to 0.4, step 0.1).
        + Output Layer: Dense layer with 10 units (for 10 rice varieties) and softmax activation.
        + Compiled with Adam optimizer, categorical_crossentropy loss, and accuracy metric.
    + Hyperparameters:
        + dense1_units: Number of units in the first dense layer.
        + dropout_rate1: Dropout rate for the first block.
        + dense2_units: Number of units in the second dense layer.
        + dropout_rate2: Dropout rate for the second block.
        + learning_rate: Optimizer learning rate (0.01, 0.001, 0.0005, 0.0001).
    + Callbacks:
        + EarlyStopping: Stops training if val_accuracy doesn’t improve for 5 epochs, restoring the best weights.
        + ReduceLROnPlateau: Reduces learning rate by half if val_loss doesn’t improve for 2 epochs (minimum 1e-6).
    + Tuner:
        + Uses BayesianOptimization to search for the best hyperparameters.
        + Optimizes for val_accuracy over 20 trials.
        + Stores results in kt_bayesian/rice_variety_tuning.
    + Output: Displays the search space summary, detailing the tunable hyperparameters and their ranges.

In [7]:
from tensorflow.keras.layers import (BatchNormalization, Activation)

def build_model(hp):
    base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Dense Block 1
    x = Dense(
        units=hp.Int('dense1_units', min_value=256, max_value=1024, step=128)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(hp.Float('dropout_rate1', min_value=0.2, max_value=0.5, step=0.1))(x)

    # Dense Block 2
    x = Dense(
        units=hp.Int('dense2_units', min_value=64, max_value=512, step=64)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(hp.Float('dropout_rate2', min_value=0.1, max_value=0.4, step=0.1))(x)

    # Output Layer
    outputs = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 5e-4, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1,
    min_lr=1e-6
)

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='kt_bayesian',
    project_name='rice_variety_tuning'
)

tuner.search_space_summary()

Search space summary
Default search space size: 5
dense1_units (Int)
{'default': None, 'conditions': [], 'min_value': 256, 'max_value': 1024, 'step': 128, 'sampling': 'linear'}
dropout_rate1 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense2_units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': 'linear'}
dropout_rate2 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.4, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0005, 0.0001], 'ordered': True}


## Fine-Tune Search

### Fine-Tune Search
This cell performs hyperparameter tuning using the `BayesianOptimization` tuner and retrieves the best hyperparameters.

In [ ]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

Trial 6 Complete [00h 26m 31s]
val_accuracy: 0.6735876202583313

Best val_accuracy So Far: 0.9159826040267944
Total elapsed time: 07h 16m 48s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
768               |1024              |dense1_units
0.4               |0.4               |dropout_rate1
64                |384               |dense2_units
0.2               |0.1               |dropout_rate2
0.001             |0.0005            |learning_rate

Epoch 1/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 277s 401ms/step - accuracy: 0.3702 - loss: 1.8940 - val_accuracy: 0.6688 - val_loss: 1.2252 - learning_rate: 0.0010
Epoch 2/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 258s 387ms/step - accuracy: 0.4391 - loss: 1.6446 - val_accuracy: 0.3211 - val_loss: 2.0007 - learning_rate: 0.0010
Epoch 3/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.4861 - loss: 1.4938
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
666/666 ━━━━━━━━━━━━━━━━━━━━ 261s 392ms/ste

## Train Model

### Train Model
This cell builds and trains the model using the best hyperparameters found from tuning.

In [8]:
best_hps = HyperParameters()
best_hps.Fixed('dense1_units', 1024)
best_hps.Fixed('dropout_rate1', 0.4)
best_hps.Fixed('dense2_units', 384)
best_hps.Fixed('dropout_rate2', 0.1)
best_hps.Fixed('learning_rate', 0.0005)

# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=[early_stopping, reduce_lr]
)

best_model.save("best_rice_variety_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 338s 455ms/step - accuracy: 0.3656 - loss: 1.9056 - val_accuracy: 0.4128 - val_loss: 1.7877 - learning_rate: 5.0000e-04
Epoch 2/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 255s 383ms/step - accuracy: 0.4716 - loss: 1.5496 - val_accuracy: 0.5031 - val_loss: 1.5178 - learning_rate: 5.0000e-04
Epoch 3/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 260s 389ms/step - accuracy: 0.5075 - loss: 1.4357 - val_accuracy: 0.6803 - val_loss: 1.1118 - learning_rate: 5.0000e-04
Epoch 4/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 257s 386ms/step - accuracy: 0.5362 - loss: 1.3436 - val_accuracy: 0.5983 - val_loss: 1.1837 - learning_rate: 5.0000e-04
Epoch 5/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 258s 388ms/step - accuracy: 0.5545 - loss: 1.2710 - val_accuracy: 0.7117 - val_loss: 0.8887 - learning_rate: 5.0000e-04
Epoch 6/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 259s 389ms/step - accuracy: 0.5873 - loss: 1.1672 - val_accuracy: 0.7141 - val_loss: 0.9397 - learning_rate: 5.0000e-04
Epoch 7/25
666/666 ━━━━━━━━━━━━━━━━━━━━ 